In [2]:
import zipfile 
import pysal as ps
%matplotlib inline
import numpy as np
import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import requests
import os
import geopandas as gpd
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import matplotlib.pylab as pl
from matplotlib import colors
import shapely
import seaborn as sns
import gzip
import mapreduce as mr
import csv

In [9]:
## start by filtering by work place, we are not interest in residences or jobs outside of NYC
def mapper1(row):
    if row['h_geocode'][:5] == '06067':
        yield (row['w_geocode'], row['h_geocode'], int(row['S000']))
    #yield (row['w_geocode'], row['h_geocode'], int(row['S000']))               
#fileUrl = 'https://lehd.ces.census.gov/data/lodes/LODES7/ca/od/ca_od_main_JT00_2014.csv.gz' ###LODES 2014 California State
#filebuffer = urllib.urlretrieve(fileUrl, filename='ca_od_main_JT00_2014.csv.gz')
with gzip.open('Data/ca_od_main_JT00_2014.csv.gz', 'rb') as f:
    reader = csv.DictReader(f)
    output1 = list(mr.run(reader, mapper1))
print len(output1)
with open('Data/Sacramento_ca_od_main_JT00_2014.csv', 'w') as f:
    wr = csv.writer(f, quoting = csv.QUOTE_ALL)
    wr.writerow(["w_geocode", "h_geocode", "S000", "w_track", "h_track",])
    map(wr.writerow,output1)
    
output1[:10]

539673


[('060014001001065', '060670051021011', 1),
 ('060014001001068', '060670055081002', 1),
 ('060014001001068', '060670091111013', 1),
 ('060014002001009', '060670074152012', 1),
 ('060014002001018', '060670003001011', 1),
 ('060014002001018', '060670015001009', 1),
 ('060014002001018', '060670035021015', 1),
 ('060014002001018', '060670040012015', 1),
 ('060014002001018', '060670040081022', 1),
 ('060014002001018', '060670055091001', 1)]

In [19]:
def mapblocks(row):
    yield (row['h_geocode'],int(row['S000']))
    
def reduceblocks(counts, pair):
    counts[pair[0]] = counts.get(pair[0], 0)+pair[1]
    return counts

with open('Data/Sacramento_ca_od_main_JT00_2014.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output2 = sorted(reduce(reduceblocks,list(mr.run(reader,mapblocks)), {}).items())
print len(output2)
output2[:10]

15206


[('060670001001004', 651),
 ('060670001001005', 30),
 ('060670001001006', 11),
 ('060670001001007', 9),
 ('060670001001008', 42),
 ('060670001001009', 17),
 ('060670001001010', 7),
 ('060670001001011', 26),
 ('060670001001012', 66),
 ('060670001001013', 34)]

In [20]:
sacramentoCB = gpd.GeoDataFrame.from_file('Data/SacramentoCensusBlock.geojson')
sacramentoCB.head()


,ALAND10,AWATER10,BLOCKCE10,COUNTYFP10,FUNCSTAT10,GEOID10,INTPTLAT10,INTPTLON10,MTFCC10,NAME10,STATEFP10,TRACTCE10,UACE10,UATYP10,UR10,geometry,id
0,122533,0,3016,067,S,060670067023016,+38.6257479,-121.4519923,G5040,Block 3016,06,006702,77068,U,U,"POLYGON ((-121.452623 38.625998, -121.452562 3...",0
1,5937,0,2018,067,S,060670066002018,+38.6299084,-121.4466320,G5040,Block 2018,06,006600,77068,U,U,"POLYGON ((-121.446116 38.629683, -121.446224 3...",1
2,8070,0,2007,067,S,060670066002007,+38.6326845,-121.4466952,G5040,Block 2007,06,006600,77068,U,U,"POLYGON ((-121.446076 38.633048, -121.446076 3...",2
3,28706,0,1021,067,S,060670036001021,+38.5342994,-121.4802584,G5040,Block 1021,06,003600,77068,U,U,"POLYGON ((-121.479199 38.533645, -121.481371 3...",3
4,41010,0,1019,067,S,060670037001019,+38.5373652,-121.4743151,G5040,Block 1019,06,003700,77068,U,U,"POLYGON ((-121.473216 38.535408, -121.473449 3...",4


In [21]:
sacramentoCB['LODES'] = 0 
def pplhouses(block_Count):
    sacramentoCB.loc[sacramentoCB.GEOID10 == block_Count[0],'LODES'] = block_Count[1]
    return
map(pplhouses,output2)
print 'done'

done


In [22]:
sacramentoCB.head()

,ALAND10,AWATER10,BLOCKCE10,COUNTYFP10,FUNCSTAT10,GEOID10,INTPTLAT10,INTPTLON10,MTFCC10,NAME10,STATEFP10,TRACTCE10,UACE10,UATYP10,UR10,geometry,id,LODES
0,122533,0,3016,067,S,060670067023016,+38.6257479,-121.4519923,G5040,Block 3016,06,006702,77068,U,U,"POLYGON ((-121.452623 38.625998, -121.452562 3...",0,14
1,5937,0,2018,067,S,060670066002018,+38.6299084,-121.4466320,G5040,Block 2018,06,006600,77068,U,U,"POLYGON ((-121.446116 38.629683, -121.446224 3...",1,5
2,8070,0,2007,067,S,060670066002007,+38.6326845,-121.4466952,G5040,Block 2007,06,006600,77068,U,U,"POLYGON ((-121.446076 38.633048, -121.446076 3...",2,9
3,28706,0,1021,067,S,060670036001021,+38.5342994,-121.4802584,G5040,Block 1021,06,003600,77068,U,U,"POLYGON ((-121.479199 38.533645, -121.481371 3...",3,20
4,41010,0,1019,067,S,060670037001019,+38.5373652,-121.4743151,G5040,Block 1019,06,003700,77068,U,U,"POLYGON ((-121.473216 38.535408, -121.473449 3...",4,2


In [23]:
with open('Data/SacramentoCensusBlock.geojson', 'w') as f:
    f.write(sacramentoCB.to_json())